**Oreum Industries Internal Project, 2024Q1**

---

# 001_MRE_ModelA0.ipynb

### Oreum Copula Demo in `oreum_copula`

Implementation of Bayesian Copula-Based Expected Loss Cost Forecasting. 
We use highly advanced Bayesian inference techniques and a Bayesian workflow, 
specifically using the `pymc` & `arviz` ecosystem.

Here we demonstrate an E2E workflow for novel models of increasing sophistication.
We evaluate the behaviour and performance of the models throughout the workflows,
including several state-of-the-art methods unavailable to conventional
max-likelihood / machine-learning models.

**In this Notebook:**

Build & test `ModelA0`, a naive version of the core project architecture **without a copula**.
To demonstrate some of the core aspects and make the case for needing a copula.


## Contents

+ [Setup](#Setup)


+ [2. Define & Build Model, Sample Prior & Evaluate](#2.-Define-&-Build-Model-&-Sample-Prior-&-Evaluate)

+ [3. Sample Posterior & Evaluate](#3.-Sample-Posterior-&-Evaluate)

+ [4. Evaluate Posterior Parameters](#4.-Evaluate-Posterior-Parameters)

+ [5. Predict Out-of-Sample (Holdout Set)](#5.-Predict-Out-of-Sample-(Holdout-Set))

+ [6. Evaluate Predictions (Holdout Set)](#6.-Evaluate-Predictions-(Holdout-Set))



---

# Setup

## Imports

In [ ]:
import sys
from copy import deepcopy
from pathlib import Path

import arviz as az
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from pyprojroot.here import here

# prepend local project src files
module_path = here('src').resolve(strict=True)
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path))    # sys.path.append(str(module_path))

# autoreload local modules to allow dev-js
%load_ext autoreload
%autoreload 2
from oreum_core import eda
from oreum_core import model_pymc as mt

from engine import app_logger
from engine.trainer import Trainer
from model.model_a import ModelA0
from synthetic.create_copula import CopulaBuilder

import warnings  # isort:skip
warnings.simplefilter(action='ignore', category=FutureWarning)  # isort:skip

##### Notebook config

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(style='darkgrid', palette='muted', context='notebook', 
        rc={'savefig.dpi':300, 'figure.figsize': (12, 3)})

log = app_logger.get_logger('000_Example_ModelA0', notebook=True)
_ = app_logger.get_logger('oreum_core', notebook=True)
# _ = app_logger.get_logger('py.warnings', notebook=True)

## Local Functions and Global Vars

In [ ]:
RSD = 42
rng = np.random.default_rng(seed=RSD)

## Data Connections and Helper Objects

In [ ]:
figio = eda.FigureIO(here(Path('plots')).resolve(strict=True))

---

---

# 0. Synthesize Dataset

See `000_MRE_EDA.ipynb` for details

In [ ]:
cb = CopulaBuilder()
df_all = cb.create(nobs=120)
print(cb.ref_vals)
perm = rng.permutation(df_all.index.values)
df_train = df_all.loc[perm[:100]]
df_holdout = df_all.loc[perm[100:]]
# eda.describe(df_train, nobs=3, get_counts=False)
eda.display_ht(df_train)

---

---

# 1. Model Description

## 2.0 Model Spec
### Estimate Marginals

#### M0

\begin{align}
    \beta_{m0}^{j1} &\sim \text{Normal}(\mu, \sigma) \\
    \sigma_{m0} &\sim \text{InverseGamma}(\alpha, \beta) \\
    \mathfrak{m0}_{y}^{j0} &\sim \text{LogNormal}(\mu=\beta_{m0}^{T}\vec{x}_{y}^{j0}, \sigma=\sigma_{m0}) \\
\end{align}


#### M1

\begin{align}
    \beta_{m1}^{j1} &\sim \text{Normal}(\mu, \sigma) \\
    \sigma_{m1} &\sim \text{InverseGamma}(\alpha, \beta) \\
    \mathfrak{m1}_{y}^{j1} &\sim \text{LogNormal}(\mu=\beta_{m1}^{T}\vec{x}_{y}^{j1}, \sigma=\sigma_{m1}) \\
\end{align}

Where:

+ We use the same distribution family for each marginal (a LogNormal), 
  because we want to keep this example simple
+ We indicate and observations as $y$ e.g. $\vec{x}_{y}^{j0}$
+ The linear models for the regression on each LogNormal marginal are exponentiated 
  for convenience, so that $\beta_{m}^{T}\vec{x}^{j}$ becomes the median value
+ Features in $\vec{x}^{j1}$ and $\vec{x}^{j2}$ are each chosen from the full 
  set of features $j \in m$
+ In our synthetic data here, we actually have no features $m$, so $j0$ and $j1$
  are both intercept-only





### ~~Estimate Copula~~

This model does not fit a copula to the marginals. See NBs 101 and 102 for detail


## 1.3 Transform observations for linear sub-models

Normally, we would use `patsy` to transform the "raw" (synthetic) data created above into two separate design-matrices (for marginal 1 and marginal 2) each according to a linear model. This is total overkill here, so we'll keep life much simpler.

We have no features for `m0` and `m1`, so the linear sub-models have to be intercept-only. Here we must also forget anything we know about the construction of the synthetic data, this is all we have.

### 1.3.1 Train set (in-sample)

In [ ]:
eda.describe(df_train[['m0', 'm1']])

In [ ]:
dfx_m0_train = df_train[['m0']].copy()
dfx_m0_train['intercept'] = 1.0
eda.display_ht(dfx_m0_train)

In [ ]:
dfx_m1_train = df_train[['m1']].copy()
dfx_m1_train['intercept'] = 1.0
eda.display_ht(dfx_m1_train)

### 1.3.2 holdout set (out-of-sample)

In [ ]:
eda.describe(df_holdout[['m0', 'm1']])

In [ ]:
dfx_m0_holdout = df_holdout[['m0']].copy()
dfx_m0_holdout['intercept'] = 1.0

In [ ]:
dfx_m1_holdout = df_holdout[['m1']].copy()
dfx_m1_holdout['intercept'] = 1.0

---

---

# 2. Define & Build Model, Sample Prior & Evaluate

#### 2.0.1 Build helper objects

In [ ]:
fqns = dict(
    fp_mdl=here(Path('data', 'models', 'mdla')).resolve(strict=True),
    fp_plots = here(Path('plots')).resolve(strict=True),
    fn_mdl='idata_mdla0',
)
trainer = Trainer(fqns=fqns)

#### 2.0.2 Sanity-check Lognorm Implementations `scipy` vs `pymc`

In [ ]:
mt.sanity_check_lognorm(cb.ref_vals['m0_params']['mu'], 
                        cb.ref_vals['m1_params']['sigma'])

## 2.1 Build Model Object

**NOTE:** Model imported from `src.model.copula.model_a.py` where it is fully defined

In [ ]:
mdl = ModelA0(obs_m0=dfx_m0_train, obs_m1=dfx_m1_train)
mdl.build()
_ = [display(Markdown(s)) for s in mt.print_rvs(mdl)]

In [ ]:
fqn = trainer.mdlio.write_graph(mdl)  # output model graph to prove built
f = eda.display_image_file(fqn, 
    title=f'Model architecture: {mdl.name} {mdl.version}', figsize=(12, 6))

##### Quick pass with [`model.debug()`](https://github.com/pymc-devs/pymc/blob/5f29b255127088abc552079fd03c40eb19d83bdd/pymc/model/core.py#L1739) and [`pymc.testing.assert_no_rvs`](https://github.com/pymc-devs/pymc/blob/c3f93bad3db7c34c12e4c51e1e7fb88f62c97020/pymc/testing.py#L952)

In [ ]:
print('Random:\n')
mdl.model.debug(fn='random', verbose=True)

# known failure for potentials: https://github.com/pymc-devs/pymc/issues/6966
print('logP:\n')
mdl.model.debug(fn='logp', verbose=True)

from pymc.testing import assert_no_rvs

assert_no_rvs(mdl.model.logp())

## 2.2 Sample Prior Predictive & Evaluate

In [ ]:
mdl.sample_prior_predictive(var_names=mdl.rvs_marg + mdl.rvs_ppc, replace=True)       
mdl.idata

### 2.2.1 Hyperpriors on Marginals for Feature Regression

In [ ]:
# get ref values from copula builder
refs = [cb.ref_vals['m0_params']['mu'], cb.ref_vals['m0_params']['sigma'], 
        cb.ref_vals['m1_params']['mu'], cb.ref_vals['m1_params']['sigma']
        ]
f = mt.facetplot_krushke(mdl=mdl, group='prior', txtadd='hyperpriors on marginals',
        rvs=mdl.rvs_marg, ref_vals=refs, m=2)

**Observe:**

+ Priors contain the reference values, seems reasonable

### 2.2.2 ~~Copula Structure~~

No copula in this naive model

### 2.2.3 Plot ECDF

In [ ]:
_ = mt.plot_ppc(mdl, group='prior', data_pairs={'yhat':'yhat'}, 
                flatten=['oid'], observed_rug=True)

**Observe:**

+ Terrible fit as expected, but the range is valid

---

---

# 3. Sample Posterior & Evaluate

## 3.1 Sample Posterior

In [ ]:
# if RELOAD_IDATA:
#     mdl.update_idata(trainer.mdlio.read_idata(fqn=trainer.fqns['fqn_idata']))
# else:
#     mdl.sample()
#     mdl.sample_posterior_predictive(store_ppc=True, ppc_insample=True, 
#                                     var_names=mdl.rvs_ppc)
#     trainer.mdlio.write_idata(mdl=mdl)

# mdl.idata

## 3.2 View Diagnostics

### 3.2.1 Plot Posterior Traces

In [ ]:
f = mt.plot_trace(mdl=mdl, rvs=mdl.rvs_marg, kind='trace')

**Observe:**
    
+ Traces appear reasonably well-mixed
+ Posterior distributions reasonably central

### 3.2.2 Summarize Posterior

In [ ]:
mt.get_summary(mdl=mdl, rvs=mdl.rvs_marg)

**Observe:**

+ Parameters are all fairly well-behaved: `ess_bulk` is good, `r_hat` is good

### 3.2.3 Plot Energy

In [ ]:
_ = mt.plot_energy(mdl=mdl)

**Observe:**

+ BFMI above 0.3, so [apparently reasonable](https://arxiv.org/pdf/1701.02434.pdf)
+ A little messy though, not symmetric

## 3.3 Evaluate In-Sample PPC

### 3.3.1 Visually Compare In-Sample Predictions to Observations

In [ ]:
# df_train[['m0', 'm1']].describe().T

In [ ]:
# eda.display_ht(mt.get_summary(mdl=mdl, rvs=mdl.rvs_ppc))

In [ ]:
# plot the real observed again
f = eda.plot_joint_numeric(data=df_train, ft0='m0', ft1='m1', kind='kde+scatter', colori=2, 
            subtitle='Observed Marginals with Copula Correlation')

### 3.3.1 View In-Sample PPC Joint `yhat0`, `yhat1`

#### Mean

In [ ]:
rv = 'yhat'
cols = mdl.coords[f'{rv}_nm']
x = az.extract(mdl.idata, group='posterior_predictive', var_names=rv).values
dfp_y = pd.DataFrame(np.nanmean(x, axis=2), columns=cols)
f = eda.plot_joint_numeric(data=dfp_y, ft0=cols[0], ft1=cols[1], kind='kde+scatter', 
            colori=2, subtitle='In-Sample Posterior Predictive Marginals (mean)')

**Observe:**

+ Joint distribution looks spherical - does not have the covariance structure of
  the actual data

#### Quantiles

In [ ]:
qs = [0.03, 0.25, 0.5, 0.75, 0.97]
dfp_q = pd.DataFrame(np.quantile(x, qs, axis=2).reshape(len(qs)*mdl.n, 2), columns=cols)
dfp_q['q'] = np.repeat([f'{q}'  for q in qs], mdl.n)
f = eda.plot_joint_numeric(data=dfp_q, ft0=cols[0], ft1=cols[1], hue='q', kind='kde',
            legendpos='lower right',
            subtitle='In-Sample Posterior Predictive Marginals (quantiles)')

**Observe**

+ The PPC quantile range covers the range of the observed data quite well
+ The extremes quantiles are still very tightly grouped - variance well managed?

### 3.3.2 Evaluate via ECDF Plot (In-Sample)

In [ ]:
_ = mt.plot_ppc(mdl, group='posterior', data_pairs={'yhat': 'yhat'}, 
                flatten=['oid'], observed_rug=True)

**Observe:**

+ Bulk: reasonably good fit
+ Tails: reasonable fit but heavier than observed

### 3.3.3 Evaluate via LOO-PIT Plot (In-Sample)

In [ ]:
_ = mt.plot_loo_pit(mdl, data_pairs={'yhat':'yhat'})

**Observe**

+ Very slight overdispersion, but otherwise a pretty good fit

### 3.3.4 ~~Compare Log-Likelihood vs other models~~

No other models yet

---

---

# 4. Evaluate Posterior Parameters

Also in this demo notebook we can evaluate parameter recovery

## 4.1 Hyperpriors on Marginals for Feature Regression

### 4.1.1 Univariate

In [ ]:
# get ref values from copula builder
refs = [cb.ref_vals['m0_params']['mu'], cb.ref_vals['m1_params']['mu'], 
        cb.ref_vals['m0_params']['sigma'], cb.ref_vals['m1_params']['sigma']
        ]
f = mt.facetplot_krushke(mdl=mdl, txtadd='hyperpriors on marginals',
        rvs=mdl.rvs_marg, ref_vals=refs, m=2, hdi_prob=0.50)

**Observe:**

+ Parameter recovery looks good for `m0_b`, `m1_b`: reference values fall into the 50% HDI of the posterior estimates
+ Parameter recovery looks good for `s1`, but not `s0`

### 4.1.2 Bivariate Pairs

In [ ]:
refsd = {
    'm0_b\nintercept': cb.ref_vals['m0_params']['mu'], 
    'm1_b\nintercept': cb.ref_vals['m1_params']['mu'],
    'm_s\ns0': cb.ref_vals['m0_params']['sigma'], 
    'm_s\ns1' :cb.ref_vals['m1_params']['sigma']
    }
f = mt.pairplot_corr(mdl=mdl, rvs=mdl.rvs_marg, ref_vals=refsd)

**Observe:**

+ Another view of the parameter recovery, which seems:
  + good for `m0_b` and `m1_b`
  + reasobale for `m_s`
+ But no correlation between `m0_b` and `m1_b`

## 4.2 ~~Copula Structure~~

---

---

# 5. Predict Out-of-Sample (Holdout Set)

## 5.1 Sample PPC holdout

In [ ]:
# replace obs with dfx_pholdout and build
mdl.replace_obs({'obs_m0': dfx_m0_holdout, 'obs_m1':dfx_m1_holdout})
mdl.build()

In [ ]:
fqn = trainer.mdlio.write_graph(mdl, txtadd='holdout')  # output model graph to prove rebuilt
f = eda.display_image_file(fqn, 
    title=f'Rebuilt model architecture for holdout: {mdl.name} {mdl.version}', figsize=(12, 6))

In [ ]:
idata_holdout = mdl.sample_posterior_predictive(store_ppc=False, 
        ppc_insample=True,  # hack to ensure output is named posterior_predictive so we can do EDCF
        var_names=mdl.rvs_ppc)

GROUP = 'posterior_predictive'  # predictions

idata_holdout

## 5.2 Plot holdouts

Calling back to **1.2.6 Compare the Impact on Joint Distribution** we might consider the PPC Joint Distribution here too. We will keep with the theme that this might represent a Expected Loss Cost $E_{loss}$

NOTE:

+ Within each marginal, the samples for the posterior parameters of a submodel 
  are coherent across the joint posterior parameter space
+ This means for example, that within parameters `yhat0` etc, each of 
  samples `0, 1, 2, 3 ... j` gives us the full state of the model at that point 
  in the Markov chain
+ So, we use all parameter values at each sample `0, 1, 2, 3 ... j`, to get 
  `j` estimates of the posterior parameter values, and thus the predictions
+ We usually set `j = 2000`, so we have $2000$ predictions for each observation

In [ ]:
yhat = az.extract(idata_holdout, group=GROUP, var_names=mdl.rvs_ppc).values
yhat_eloss = np.product(yhat, axis=1)
yhat_eloss.shape

### 5.2.1 Plot holdout Full Set Distribution

In [ ]:
df_yhat_eloss = pd.DataFrame(yhat_eloss, index=dfx_m0_holdout.index)
dfm_yhat_eloss = df_yhat_eloss.reset_index().melt(
                        id_vars='index', value_name='yhat', var_name='sample')
eda.display_ht(df_yhat_eloss)

In [ ]:
f = eda.plot_estimate(dfm_yhat_eloss, len(df_yhat_eloss), yhat='yhat')

In [ ]:
f = eda.plot_estimate(dfm_yhat_eloss, len(df_yhat_eloss), yhat='yhat', kind='exceedance')

**Observe:**

+ Quite a long tail, high mean, doesn't seem unreasonable

### 5.2.2 Plot holdout Individual Observations

In [ ]:
mn_pt_kws=dict(markerfacecolor='w', markeredgecolor='#333333', marker='d', markersize=8)
box_kws=dict(kind='box', sym='', orient='h', showmeans=True, whis=(3, 97), meanprops=mn_pt_kws)
nobs = len(df_yhat_eloss)
gd = sns.catplot(x='yhat', y='index', data=dfm_yhat_eloss, **box_kws, height=4, aspect=2)
_ = gd.fig.suptitle(f'Individual Distribution of yhat Estimate for {nobs} Observations')
_ = gd.fig.tight_layout()

#### 5.2.3 Mean Joint Distribution

In [ ]:
rv = 'yhat'
cols = mdl.coords[f'{rv}_nm']
x = az.extract(idata_holdout, group=GROUP, var_names=rv).values
dfp_y = pd.DataFrame(np.nanmean(x, axis=2), columns=cols)
f = eda.plot_joint_numeric(data=dfp_y, ft0=cols[0], ft1=cols[1], kind='kde+scatter', colori=2, 
            subtitle='holdout Out-of-Sample Posterior Predictive Marginals (mean)')

**Observe:**

+ Quite broad variance
+ But marginals and joint look reasonably close to holdout actual values

#### 5.2.4 Quantile Joint Distribution

In [ ]:
qs = [0.03, 0.25, 0.5, 0.75, 0.97]
dfp_q = pd.DataFrame(np.quantile(x, qs, axis=2).reshape(len(qs)*mdl.n, 2), columns=cols)
dfp_q['q'] = np.repeat([f'{q}'  for q in qs], mdl.n)
f = eda.plot_joint_numeric(data=dfp_q, ft0=cols[0], ft1=cols[1], hue='q', kind='kde',
            legendpos='lower right',
            subtitle='holdout Out-of-Sample Posterior Predictive Marginals (quantiles)')

**Observe**

+ Very interesting! The PPC quantile range looks reasonable

**In the real world, we have to stop here, because in a holdout scenario we dont have `y`**

---

# 6. Evaluate Predictions (Holdout Set)

**IMPORTANT NOTE** 

Strictly speaking, in a **holdout** dataset / scenario we dont have `y`, but in
this worked example Notebook we do have `y`, so we can treat this more like a 
**Holdout** dataset / scenario, and add several evaluations including:

+ Plot Summarised Predictions with overplotted Actual
+ Plot PPC ECDF
+ Plot Coverage / Calibration
+ Plot RMSE and R^2


## 6.1 Evaluate holdout Joint Distribution ($E_{loss}$)

### 6.1.1 Plot Summarised Predictions with overplotted Actual

In [ ]:
y_m0 = dfx_m0_holdout['m0'].values
y_m1 = dfx_m1_holdout['m1'].values
y_eloss = y_m0 * y_m1
df_y_eloss = pd.DataFrame({'y': y_eloss}, index=dfx_m0_holdout.index).reset_index()

In [ ]:
f = eda.plot_estimate(dfm_yhat_eloss, len(df_yhat_eloss), yhat='yhat', 
                    arroverplot=df_y_eloss['y'], txtadd='with overplotted bootstrapped Actual')
figio.write(f, fn=f'100_6.1.1_holdout_prediction_{mdl.name}')

**Observe:**

+ Now we can see the prediction from this naive model is much too high and the 
  variance is too large
+ The overplotted actual sample mean is 9.3, and the estimated mean is 16.2,
  a $16.2/9.3 \sim +74\%$ overestimate!


### 6.1.2 Plot PPC EDCF

In [ ]:
# hacky correct for sample_posterior_predictive not creating observed data 
# (because model observed_RVs is empty)
if 'observed_data' not in idata_holdout.groups():
    idata_holdout.add_groups(observed_data=deepcopy(mdl.idata.observed_data))

idata_holdout

In [ ]:
_ = mt.plot_ppc(mdl, idata=idata_holdout, group='posterior', insamp=False,
                data_pairs={'yhat': 'yhat'}, flatten=['oid'], observed_rug=True)

**Observe:**

+ Bulk: reasonably good fits, but high variance
+ Tails: reasonable fit, rather heavier than observed, esp for `yhat1`

### 6.1.3 Plot Coverage

In [ ]:
df_cov_eloss = mt.calc_ppc_coverage(y_eloss, yhat_eloss.T)
f = eda.plot_coverage(df_cov_eloss)

**Observe:**

+ Reasonably well-calibrated
+ AUC: quite large

### 6.1.4: Plot RMSE and R^2

In [ ]:
rmse, rmse_pct = mt.calc_rmse(y_eloss, yhat_eloss.T)
f = eda.plot_rmse_range(rmse, rmse_pct, yhat_name='yhat_eloss')

r2 = mt.calc_bayesian_r2(y_eloss, yhat_eloss)
f = eda.plot_float_dist(r2, ['r2'], log=False)

**Observe:**

+ RMSE quite high variance
+ R^2 mean seems reasonable, but has high variance

## 6.2 Evaluate Marginal Distributions

**IMPORTANT NOTE** 

Strictly speaking, in a **holdout** dataset / scenario we dont have `y`, but in
this worked example Notebook we do have `y`, so we can treat this more like a 
**Holdout** dataset / scenario, and add several evaluations including:

+ Plot Coverage
+ Plot RMSE and R^2


### 6.2.1 Margin M0

#### 6.2.1.1 Coverage

In [ ]:
dfcov_m0 = mt.calc_ppc_coverage(y_m0, yhat[:, 0].T)
f = eda.plot_coverage(dfcov_m0)

**Observe:**

+ Looks pretty well-calibrated

#### 6.2.1.2: RMSE and R^2

In [ ]:
rmse, rmse_pct = mt.calc_rmse(y_m0, yhat[:, 0].T)
f = eda.plot_rmse_range(rmse, rmse_pct, yhat_name='m0')

r2 = mt.calc_bayesian_r2(y_m0, yhat[:, 0])
f = eda.plot_float_dist(r2, ['r2'], log=False)

**Observe:**

+ RMSE quite high variance
+ R^2 mean seems reasonable, but has high variance

### 6.2.2 Margin M1

#### 6.2.2.1 Coverage

In [ ]:
dfcov_m1 = mt.calc_ppc_coverage(y_m1, yhat[:, 1].T)
f = eda.plot_coverage(dfcov_m1)

**Observe:**

+ Also reasonably well-calibrated

#### 6.2.2.2: RMSE and R^2

In [ ]:
rmse, rmse_pct = mt.calc_rmse(y_m1, yhat[:, 1].T)
f = eda.plot_rmse_range(rmse, rmse_pct, yhat_name='m1')

r2 = mt.calc_bayesian_r2(y_m1, yhat[:, 1])
f = eda.plot_float_dist(r2, ['r2'], log=False)

**Observe:**

+ RMSE quite high variance
+ R^2 mean seems reasonable, but has high variance

---

---

# Notes

In [ ]:
%load_ext watermark
%watermark -a "jonathan.sedar@oreum.io" -udtmv -iv -p pymc,pytensor

---
**Oreum OÜ &copy; 2024**